## Tools in LlamaIndex

Defining a clear set of  tools is crucial to performance. Clear tool interfaces are easier for LLMs to use, similar to software API interface for human engineers.

`LlamaIndex` has 4 main types of tools: Function Tools, Search Engine, ToolSpecs, Utility Tools

### Function Tool:
Provides a simple way to wrap any Python function and make it available to any agent -- could be synchronous, or async, with optional `name` and `description`. The `name` and `description` are important as they help the agent understand when and how to use the tool effectively.

In [2]:
from llama_index.core.tools import FunctionTool

def get_weather(location: str) -> str:
    """Useful for getting the weather for a given location."""
    print(f'Getting weather for {location}')
    return f'The weather in {location} is sunny.'

tool = FunctionTool.from_defaults(
    get_weather, 
    name='my_weather_tool',
    description="Useful for getting the weather for a given location.",
)

tool.call("New York")

Getting weather for New York


ToolOutput(content='The weather in New York is sunny.', tool_name='my_weather_tool', raw_input={'args': ('New York',), 'kwargs': {}}, raw_output='The weather in New York is sunny.', is_error=False)

### Query Engine Tool

The Query Engine we created in `RAGUsingQueryEngine.ipynb` can easily be transformed into a tool.

In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

from dotenv import load_dotenv
load_dotenv()   # load OPENAI API key into memory

embed_model = HuggingFaceEmbedding('BAAI/bge-small-en-v1.5')

db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection('alfred')
vector_store = ChromaVectorStore(chroma_collection = chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model = embed_model
)

llm = OpenAI(model="gpt-4o-mini") 
query_engine = index.as_query_engine(
    llm=llm
)

tool = QueryEngineTool.from_defaults(
    query_engine, name = 'Alfreds Persona Query Engine', description = 'A persona of a helpful assistant'
)

await tool.acall(
    "Responds about research on the impact of AI on the future of work and society?"
)

ToolOutput(content='The impact of AI on the future of work and society is a multifaceted topic that encompasses various aspects, including changes in job dynamics, the need for new skills, and the potential for increased inequality. Research indicates that while AI can enhance productivity and create new job opportunities, it may also lead to job displacement in certain sectors. Additionally, the integration of AI into workplaces raises ethical considerations regarding privacy, bias, and decision-making processes. As AI continues to evolve, it is crucial to address these challenges through policies that promote equity and inclusion, ensuring that all communities benefit from technological advancements.', tool_name='Alfreds Persona Query Engine', raw_input={'input': 'Responds about research on the impact of AI on the future of work and society?'}, raw_output=Response(response='The impact of AI on the future of work and society is a multifaceted topic that encompasses various aspects, in

### ToolSpecs

`ToolSpecs` are collections of tools that work together harmoniously -- like a professional toolkit. For instance, an accounting agent's `ToolSpec` might elegantly integrate spreadsheet capabilities, email functionality, and calculation tools to handle financial tasks with precision and efficiency.

In [6]:
from llama_index.tools.google import GmailToolSpec

tool_spec = GmailToolSpec()
tool_spec_list = tool_spec.to_tool_list()

# view metadata of each tool, for a more detailed view
[(tool.metadata.name, tool.metadata.description) for tool in tool_spec_list]

[('load_data',
  "load_data() -> List[llama_index.core.schema.Document]\nLoad emails from the user's account."),
 ('search_messages',
  "search_messages(query: str, max_results: Optional[int] = None)\nSearches email messages given a query string and the maximum number\n        of results requested by the user\n           Returns: List of relevant message objects up to the maximum number of results.\n\n        Args:\n            query[str]: The user's query\n            max_results (Optional[int]): The maximum number of search results\n            to return.\n        "),
 ('create_draft',
  "create_draft(to: Optional[List[str]] = None, subject: Optional[str] = None, message: Optional[str] = None) -> str\nCreate and insert a draft email.\n           Print the returned draft's message and id.\n           Returns: Draft object, including draft id and message meta data.\n\n        Args:\n            to (Optional[str]): The email addresses to send the message to\n            subject (Optiona

### Model Context Protocol (MCP) in LlamaIndex

LlamaIndex allows using MCP through a [ToolSpec on the LlamaHub](https://llamahub.ai/l/tools/llama-index-tools-mcp?from=)You can simply run an MCP server and start using it as follows:


In [ ]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# assume MCP server running on 127.0.0.1:8000
mcp_client = BasicMCPClient('http://127.0.0.1:8000/sse')
mcp_tool = McpToolSpec(client=mcp_client)

# sync:
# tools = mcp_tool.to_tool_list()

# async:
tools = await mcp_tool.to_tool_list_async()

## Then use `tools` in your agent as below
# from llama_index.core.agent.workflow import FunctionAgent
# from llama_index.llms.openai import OpenAI

# agent = FunctionAgent(
#     name="Agent",
#     description="Some description",
#     llm=OpenAI(model="gpt-4o"),
#     tools=tools,
#     system_prompt="You are a helpful assistant.",
# )

# resp = await agent.run("What is the weather in Tokyo?")


### Utility Tools
In addition to the above, there are also Utility Tools that can be used to perform common tasks. For instance, 
- `OnDemandToolLoader` turns any existing `LlamaIndex` data loader (`BaseReader` class) into a tool that an agent can use. The tool can be called with all the parameters needed to trigger load_data from the data loader, along with a natural language query string. During execution, we first load data from the data loader, index it (for instance with a vector store), and then query it ‘on-demand’. All three of these steps happen in a single tool call.
- `LoadAndSearchToolSpec` takes in any existing Tool as input. As a tool spec, it implements `to_tool_list` and when that function is called, two tools are returned: one for loading data and one for searching the loaded data. The load Tool execution would call the underlying Tool, and the search Tool execution would take in a query string as input and call the underlying index.


**See LlamaHub for more**